In [ ]:
"""
@author Riordan Dervin Alfredo <riordan.alfredo@monash.edu>
"""
# Run to extract all pozyx position data
# json -> csv
%run pozyx_extraction.py -p "data/145/145.json" -o "out/pos/" -s "data/145/"

In [3]:
# install it separately https://ffmpeg.org/. Use this to re-sample audio data to 48KHz 
!ffmpeg -i "./data/145/simulation_GREEN_10-Aug-2021_15-36-35-439_audio.wav" -ar 48000 "./data/145/sim_green.wav"

^C


In [16]:
# TODO: replace audio file name!
# A script to convert speech to text, wav -> csv
%run hive_automation.py -a "data/145/sim_blue.wav" -o "out/audio-sim/145_blue.csv" -s "145" -w "1" -t "3"

start
here
 before vad
1988.9168333333334
thread_started: speech2text thread 0
thread_started: speech2text thread 1
thread_started: speech2text thread 2


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

thread_finished: speech2text thread 0
thread_finished: speech2text thread 1
thread_finished: speech2text thread 2
object


In [1]:
import pandas as pd
from datetime import datetime

colour = "BLUE"
session = "145"
pos_dir = './out/pos'
audio_sim_dir = './out/audio-sim'

In [4]:


dfp = pd.read_csv('{}/{}_{}.csv'.format(pos_dir,session,colour))
dfa = pd.read_csv('{}/{}_{}.csv'.format(audio_sim_dir,session,colour))

dfp.head()

FileNotFoundError: [Errno 2] No such file or directory: './out/pos/145_BLUE.csv'

In [5]:
res = pd.merge(dfp, dfa, on="audio time")
final = res.drop(labels=["Unnamed: 0_x", "Unnamed: 0_y", "session"], axis=1)
final['tagId'] = colour
final.head()


NameError: name 'dfp' is not defined

In [3]:

final.to_csv("./out/result/{}_{}.csv".format(session,colour), sep=',', encoding='utf-8', index=False)

# Save to front-end too!
client_dir = "../client/src/projects/hive/data/clean"
final.to_csv("{}/{}_{}.csv".format(client_dir,session,colour), sep=',', encoding='utf-8', index=False)

NameError: name 'final' is not defined

In [9]:

# combine all data
df = pd.concat(map(pd.read_csv, [
    '{}/{}_RED.csv'.format(client_dir, session), 
    '{}/{}_YELLOW.csv'.format(client_dir, session), 
    '{}/{}_BLUE.csv'.format(client_dir, session),
    '{}/{}_GREEN.csv'.format(client_dir, session)]), ignore_index=True)

df = df.sort_values(by='audio time')

In [10]:
df.to_csv("../client/src/projects/hive/data/clean/{}_all.csv".format(session), sep=',', encoding='utf-8', index=False)

df

,audio_start_timestamp,audio time,x,y,audio,tagId
0,1.628574e+09,0:00:01,1622.484799,-1751.742400,0,RED
3936,1.628574e+09,0:00:01,630.094718,-2095.770178,0,BLUE
5924,1.628574e+09,0:00:01,2327.518548,-1892.962961,0,GREEN
1948,1.628574e+09,0:00:01,159.551784,-499.523955,0,YELLOW
1,1.628574e+09,0:00:02,1630.707280,-1740.337025,0,RED
...,...,...,...,...,...,...
7910,1.628574e+09,0:33:07,1587.541845,2250.186317,1,GREEN
3934,1.628574e+09,0:33:07,6690.755365,878.175536,0,YELLOW
3935,1.628574e+09,0:33:08,6679.550870,873.704941,1,YELLOW
5923,1.628574e+09,0:33:08,4417.433140,6699.844481,0,BLUE
